# Predicting car prices for used cars using `Car Dekho` data

## Table Of Contents

In [1]:
! pip install pandas
! pip install scikit-learn
! pip install matplotlib
! pip install plotly
! pip install numpy
! pip install seaborn
! pip install statsmodels 

In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error,  mean_absolute_error

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from math import sqrt
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
## Read all data files. 
car_data = pd.read_csv("./data/car_data.csv", encoding="utf-8")
car_details = pd.read_csv("./data/car_details_v3.csv", encoding="utf-8")
car_dekho_details = pd.read_csv("./data/car_details_from_car_dekho.csv", encoding="utf-8")


In [4]:
## Lets start exploration with `car_data`
## look at top 5 rows
car_data.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [5]:
## Overall shape of the table.
car_data.shape

(301, 9)

In [6]:
## Lets look at the data types of the columns

pd.DataFrame(car_data.dtypes, columns=['DataType'])

,DataType
Car_Name,object
Year,int64
Selling_Price,float64
Present_Price,float64
Kms_Driven,int64
Fuel_Type,object
Seller_Type,object
Transmission,object
Owner,int64


In [7]:
## Notes
## Observations
### Looks like Year, Selling_Price, Present_Price, Kms_Driven are numeric and doesn't have missing data as DataType is float. 
### Fuel_Type and Seller_Type might be constant strings. 
### Not sure what Owner field means? Is it 0/1 values. 
## Required Changes
## Need to change column names to more readable lower case formats
## Need to change Year values to datetime instead of int64


In [8]:
## Find missing data
car_data.isnull().mean()

Car_Name         0.0
Year             0.0
Selling_Price    0.0
Present_Price    0.0
Kms_Driven       0.0
Fuel_Type        0.0
Seller_Type      0.0
Transmission     0.0
Owner            0.0
dtype: float64

In [9]:
## Great news! No missing data. Lets repeat these steps for other data files. 

In [10]:
## Lets start exploration with `car_data`
## look at top 5 rows
car_details.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [11]:
## Overall shape of the table.
car_details.shape

(8128, 13)

In [12]:
## Notes
### So number of rows are more here than in `car_data` not sure if the data in files are related.

In [13]:
## Lets look at the data types of the columns

pd.DataFrame(car_details.dtypes, columns=['DataType'])

,DataType
name,object
year,int64
selling_price,int64
km_driven,int64
fuel,object
seller_type,object
transmission,object
owner,object
mileage,object
engine,object


In [14]:
## Notes
## Observations
## Required Changes
### max_power, torque, and engine can be converted to int/float by removing the units. 
### fuel, seller_type and transmission can also be converted to int by one hot encoding technique. 
### owner field can be converted to number of previous owners. 

In [15]:
## Find missing data
car_details.isnull().mean()

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026452
torque           0.027313
seats            0.027190
dtype: float64

In [16]:
## unfortunately mileage, engine, max_power, torque and seats have ~2% missing data.
## We'll have to see how many records are removed of we remove this data


In [17]:
## lets check how much data we'll loose if we drop rows with missing values. 
print('number of rows with missing values: ', car_details.shape[0])
print('number of rows without missing values: ', car_details.dropna().shape[0])

print('data lost on dropna : ', ((car_details.shape[0] - car_details.dropna().shape[0])/car_details.shape[0]))


number of rows with missing values:  8128
number of rows without missing values:  7906
data lost on dropna :  0.027312992125984252


In [18]:
### so loosing a little less then 3% of the data. which I think should be ok. 


In [19]:
### Lets explore last file. 
## Lets start exploration with `car_data`
## look at top 5 rows
car_dekho_details.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner
0,Maruti 800 AC,2007,60000,70000,Petrol,Individual,Manual,First Owner
1,Maruti Wagon R LXI Minor,2007,135000,50000,Petrol,Individual,Manual,First Owner
2,Hyundai Verna 1.6 SX,2012,600000,100000,Diesel,Individual,Manual,First Owner
3,Datsun RediGO T Option,2017,250000,46000,Petrol,Individual,Manual,First Owner
4,Honda Amaze VX i-DTEC,2014,450000,141000,Diesel,Individual,Manual,Second Owner


In [20]:
car_dekho_details.shape

(4340, 8)

In [21]:
## Lets look at the data types of the columns

pd.DataFrame(car_dekho_details.dtypes, columns=['DataType'])

,DataType
name,object
year,int64
selling_price,int64
km_driven,int64
fuel,object
seller_type,object
transmission,object
owner,object


In [22]:
## From the shape and size of the data it seems like this data file has limited number of rows and types of columns.
## All three data files are not related and looks like `car_details` has the required dataset for creating model for price prediction. 

In [23]:
## Data Cleaning and feature engineering 
## We are going to work with data from `car_details_v3.csv` 

In [24]:
## look at top 5 rows
car_details.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [25]:
## Lets look at the data types of the columns
pd.DataFrame(car_details.dtypes, columns=['DataType'])

,DataType
name,object
year,int64
selling_price,int64
km_driven,int64
fuel,object
seller_type,object
transmission,object
owner,object
mileage,object
engine,object


In [26]:
## Find missing data
car_details.isnull().mean()

name             0.000000
year             0.000000
selling_price    0.000000
km_driven        0.000000
fuel             0.000000
seller_type      0.000000
transmission     0.000000
owner            0.000000
mileage          0.027190
engine           0.027190
max_power        0.026452
torque           0.027313
seats            0.027190
dtype: float64

In [27]:
## lets remove missing rows as analyzed earlier. 
car_details.dropna(inplace=True)

In [28]:
## verify our dropped data - there should be no missing values. 
car_details.isnull().mean()

name             0.0
year             0.0
selling_price    0.0
km_driven        0.0
fuel             0.0
seller_type      0.0
transmission     0.0
owner            0.0
mileage          0.0
engine           0.0
max_power        0.0
torque           0.0
seats            0.0
dtype: float64

In [29]:
## Lets look at "correlations" between our columns
car_details.corr()

,year,selling_price,km_driven,seats
year,1.000000,0.412302,-0.428548,-0.007923
selling_price,0.412302,1.000000,-0.222158,0.041617
km_driven,-0.428548,-0.222158,1.000000,0.227259
seats,-0.007923,0.041617,0.227259,1.000000


In [30]:
## Notes
### Looks like `selling_price` is positively affected by `year` and negligible negatively effected by km_driven. 
### Negligible affect with number of seats. 
### Somehow a lot of columns are ignored and by guess is we'll need to clean-up, reformat and feature engineer those. 

In [31]:
## lets explore and cleanup columns one at a time. 
## starting with name. 
car_details["name"].describe()

count                       7906
unique                      1982
top       Maruti Swift Dzire VDI
freq                         129
Name: name, dtype: object

In [32]:
##may be we can split the name into Make of the car. 

car_details["make"] = car_details["name"]
car_details["make"]  = car_details["make"].apply(lambda f: f.split(" ")[0])

In [33]:
car_details.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,make
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,Hyundai
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti


In [34]:
car_details["make"] 

0        Maruti
1         Skoda
2         Honda
3       Hyundai
4        Maruti
         ...   
8123    Hyundai
8124    Hyundai
8125     Maruti
8126       Tata
8127       Tata
Name: make, Length: 7906, dtype: object

In [35]:
## Notes
### There are way too many car Make names to do a one hot encoding. One thing we can do is add a binary flag  saying something like top_brand or not. 
top_brand_cars = car_details.groupby(["make"])["make"].count().sort_values(ascending=False).reset_index(name="count").head(20)["make"].tolist()

car_details["top_brand"] = car_details["make"].apply(lambda f: 1 if f in top_brand_cars else 0 )

In [36]:
## Lets explore year to see if any changes are needed
car_details["year"].describe()

count    7906.000000
mean     2013.983936
std         3.863695
min      1994.000000
25%      2012.000000
50%      2015.000000
75%      2017.000000
max      2020.000000
Name: year, dtype: float64

In [37]:
## Notes
### For the year right now it doesn't make sense to make any changes. its a float value, and Not sure if it makes sense in changing it to datetime year value. 

In [38]:
car_details["selling_price"].describe()

count    7.906000e+03
mean     6.498137e+05
std      8.135827e+05
min      2.999900e+04
25%      2.700000e+05
50%      4.500000e+05
75%      6.900000e+05
max      1.000000e+07
Name: selling_price, dtype: float64

In [39]:
car_details["km_driven"].describe()

count    7.906000e+03
mean     6.918866e+04
std      5.679230e+04
min      1.000000e+00
25%      3.500000e+04
50%      6.000000e+04
75%      9.542500e+04
max      2.360457e+06
Name: km_driven, dtype: float64

In [40]:
## Notes
### Same for selling_price and km_driven no data changes required right now. May be normalization/stardardization but we'll visit it later on. 

In [41]:
## Lets explore fuel colum
car_details["fuel"].describe()

count       7906
unique         4
top       Diesel
freq        4299
Name: fuel, dtype: object

In [42]:
car_details["fuel"].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [43]:
## Notes
### So around 4 unique values which should be easy to convert into one hot encoding. 
car_details.groupby(["fuel"])["fuel"].count()

fuel
CNG         52
Diesel    4299
LPG         35
Petrol    3520
Name: fuel, dtype: int64

In [44]:
fuel_ohe = pd.get_dummies(car_details["fuel"]).rename(columns={
     "CNG":"cng",
     "Diesel":"diesel",
     "LPG":"lpg",
     "Petrol":"petrol",
})
fuel_ohe

,cng,diesel,lpg,petrol
0,0,1,0,0
1,0,1,0,0
2,0,0,0,1
3,0,1,0,0
4,0,0,0,1
...,...,...,...,...
8123,0,0,0,1
8124,0,1,0,0
8125,0,1,0,0
8126,0,1,0,0


In [45]:
car_details = pd.concat([car_details, fuel_ohe], axis=1)
car_details

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,make,top_brand,cng,diesel,lpg,petrol
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti,1,0,1,0,0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda,1,0,1,0,0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda,1,0,0,0,1
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,Hyundai,1,0,1,0,0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai i20 Magna,2013,320000,110000,Petrol,Individual,Manual,First Owner,18.5 kmpl,1197 CC,82.85 bhp,113.7Nm@ 4000rpm,5.0,Hyundai,1,0,0,0,1
8124,Hyundai Verna CRDi SX,2007,135000,119000,Diesel,Individual,Manual,Fourth & Above Owner,16.8 kmpl,1493 CC,110 bhp,"24@ 1,900-2,750(kgm@ rpm)",5.0,Hyundai,1,0,1,0,0
8125,Maruti Swift Dzire ZDi,2009,382000,120000,Diesel,Individual,Manual,First Owner,19.3 kmpl,1248 CC,73.9 bhp,190Nm@ 2000rpm,5.0,Maruti,1,0,1,0,0
8126,Tata Indigo CR4,2013,290000,25000,Diesel,Individual,Manual,First Owner,23.57 kmpl,1396 CC,70 bhp,140Nm@ 1800-3000rpm,5.0,Tata,1,0,1,0,0


In [46]:
## Notes
### since this is car data from India my intuition tells me we can also simplify fuel type by whether its petrol or not. But we can verify it later on. 

In [47]:
## Next is seller_type
car_details["seller_type"].describe()

count           7906
unique             3
top       Individual
freq            6563
Name: seller_type, dtype: object

In [48]:
car_details.groupby(["seller_type"])["seller_type"].count()

seller_type
Dealer              1107
Individual          6563
Trustmark Dealer     236
Name: seller_type, dtype: int64

In [49]:
## Notes
### A bit surprised to see seller type dealer , but the numbers kind of make sense. 
### For now will convert it into a boolean on weather a seller is individual or not.  
### Not sure what a Trustmark Dealer is, may be a more verified dealer, but we don't have enough records to make a huge difference. 

In [50]:
## creating new individual_dealer column to 1 if seller_type is Individual and 0 otherwise. 
car_details["individual_dealer"] = car_details["seller_type"].apply(lambda f: 1 if f == "Individual" else 0 )

In [51]:
## lets verify our update, numbers should align with `seller_type` group by counts
car_details.groupby(["individual_dealer"])["individual_dealer"].count()

individual_dealer
0    1343
1    6563
Name: individual_dealer, dtype: int64

In [52]:
## Next lets deal with transmission
car_details["transmission"].describe()

count       7906
unique         2
top       Manual
freq        6865
Name: transmission, dtype: object

In [53]:
car_details.groupby(["transmission"])["transmission"].count()

transmission
Automatic    1041
Manual       6865
Name: transmission, dtype: int64

In [54]:
## Notes
### Transmission types were as expected, based on my experience most of the cars in India are manual. Won't be surpirsed if this has a very low correlation with selling price
### Will create a new feature manual_transmision and set it to 1 or 0 based on transmission value. 

In [55]:
## creating new manual_transmission column to 1 if seller_type is Individual and 0 otherwise. 
car_details["manual_transmission"] = car_details["transmission"].apply(lambda f: 1 if f == "Manual" else 0 )

In [56]:
## lets verify the feature, the counts should match transmission counts
car_details.groupby(["manual_transmission"])["manual_transmission"].count()

manual_transmission
0    1041
1    6865
Name: manual_transmission, dtype: int64

In [57]:
## Now lets look Owner column
car_details["owner"].describe()

count            7906
unique              5
top       First Owner
freq             5215
Name: owner, dtype: object

In [58]:
car_details.groupby(["owner"])["owner"].count()

owner
First Owner             5215
Fourth & Above Owner     160
Second Owner            2016
Test Drive Car             5
Third Owner              510
Name: owner, dtype: int64

In [59]:
## Notes
### This is interesting, my intuition tells me that selling price will differ as number of owners increases but shouldn't make a diff after 3 or 4th owner. 
### I can may be combine third and above with test drive and then do one hot encoding. 

In [60]:
third_and_above_owner_cond = (car_details["owner"] != "First Owner") & (car_details["owner"] != "Second Owner")
car_details.loc[third_and_above_owner_cond, ["owner"]] = "Third & Above"

In [61]:
## Lets verify our changes
car_details.groupby(["owner"])["owner"].count()

owner
First Owner      5215
Second Owner     2016
Third & Above     675
Name: owner, dtype: int64

In [62]:
## Lets convert this to one hot encoding using the get_dummies method. 
owner_ohe = pd.get_dummies(car_details["owner"]).rename(columns={
     "First Owner": "one_owner",
     "Second Owner": "two_owners",
     "Third & Above": "above_three_owners"
})
owner_ohe

,one_owner,two_owners,above_three_owners
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,1,0,0
...,...,...,...
8123,1,0,0
8124,0,0,1
8125,1,0,0
8126,1,0,0


In [63]:
## update the car_details_table
car_details = pd.concat([car_details, owner_ohe], axis=1)
car_details.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,make,top_brand,cng,diesel,lpg,petrol,individual_dealer,manual_transmission,one_owner,two_owners,above_three_owners
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,Maruti,1,0,1,0,0,1,1,1,0,0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,Skoda,1,0,1,0,0,1,1,0,1,0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third & Above,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,Honda,1,0,0,0,1,1,1,0,0,1
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0,Hyundai,1,0,1,0,0,1,1,1,0,0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0,Maruti,1,0,0,0,1,1,1,1,0,0


In [64]:
## next lets look at mileage
car_details["mileage"].describe()

count          7906
unique          393
top       18.9 kmpl
freq            225
Name: mileage, dtype: object

In [65]:
## Notes 
### its a string column because of the unit.
### We can fix this in two steps,
#### 1. trim the units
#### 2. convert the column to float. 

In [66]:
## Trim the last unit
car_details["mileage"] = car_details["mileage"].apply(lambda f: f.split(" ")[0])

## Convert into float 
car_details["mileage"] = car_details["mileage"].astype(float)

In [67]:
## quickly verify the converstion.
car_details["mileage"]

0       23.40
1       21.14
2       17.70
3       23.00
4       16.10
        ...  
8123    18.50
8124    16.80
8125    19.30
8126    23.57
8127    23.57
Name: mileage, Length: 7906, dtype: float64

In [68]:
## Next lets look at the engine
car_details["engine"].describe()

count        7906
unique        121
top       1248 CC
freq         1017
Name: engine, dtype: object

In [69]:
car_details.groupby(["engine"])["engine"].count()

engine
1047 CC      18
1061 CC      59
1086 CC     138
1120 CC      53
1150 CC       7
1172 CC      10
1186 CC      61
1193 CC      45
1194 CC      18
1196 CC      92
1197 CC     832
1198 CC     182
1199 CC     271
1248 CC    1017
1298 CC      41
1299 CC       3
1339 CC       1
1341 CC       9
1343 CC       3
1364 CC      54
1368 CC      21
1373 CC      43
1388 CC       9
1390 CC       3
1396 CC     321
1399 CC     121
1405 CC     113
1422 CC       1
1451 CC       3
1461 CC     185
1462 CC      10
1489 CC       1
1493 CC     142
1495 CC      20
1496 CC      49
1497 CC     181
1498 CC     375
1499 CC      18
1527 CC       4
1582 CC     156
1586 CC      20
1590 CC       1
1591 CC      48
1595 CC       3
1596 CC       9
1597 CC       6
1598 CC     101
1599 CC      18
1781 CC       2
1794 CC      14
1795 CC       2
1796 CC       3
1797 CC       6
1798 CC      21
1799 CC      13
1896 CC      17
1948 CC      12
1950 CC       1
1956 CC      24
1968 CC      44
1969 CC      35
1984 CC      32
1

In [70]:
## Notes
### So number and types of engines are a lot to convert it into categorical value. 
### We'll convert it into int by removing CC and changing column type. 

In [71]:
## remove CC from engine values. 
car_details["engine"] = car_details["engine"].apply(lambda f: f.split(" ")[0])

## change the type to int
car_details["engine"] = car_details["engine"].astype(int)

## verify the changes. 
car_details["engine"]

0       1248
1       1498
2       1497
3       1396
4       1298
        ... 
8123    1197
8124    1493
8125    1248
8126    1396
8127    1396
Name: engine, Length: 7906, dtype: int64

In [72]:
## next lets look at max_power
car_details["max_power"].describe()

count       7906
unique       320
top       74 bhp
freq         377
Name: max_power, dtype: object

In [73]:
## safe to say we can similar approach for max_power as well. 
## Remove Unit
## Convert to float. 
## remove CC from engine values. 
car_details["max_power"] = car_details["max_power"].apply(lambda f: f.split(" ")[0])

## change the type to int
car_details["max_power"] = car_details["max_power"].astype(float)

## verify the changes. 
car_details["max_power"]


0        74.00
1       103.52
2        78.00
3        90.00
4        88.20
         ...  
8123     82.85
8124    110.00
8125     73.90
8126     70.00
8127     70.00
Name: max_power, Length: 7906, dtype: float64

In [74]:
## next is torque
car_details["torque"].describe()

count               7906
unique               441
top       190Nm@ 2000rpm
freq                 530
Name: torque, dtype: object

In [75]:
car_details.groupby(["torque"])["torque"].count()

torque
10.2@ 2,600(kgm@ rpm)            1
10.4@ 3,200(kgm@ rpm)            2
10.7@ 2,500(kgm@ rpm)            3
10.7@ 2,600(kgm@ rpm)            2
100Nm@ 2700rpm                   8
101Nm@ 3000rpm                  38
102Nm@ 2600rpm                   5
102Nm@ 4000rpm                  11
103Nm@ 4500rpm                   6
104Nm@ 3100rpm                   5
104Nm@ 4000rpm                  51
104Nm@ 4400rpm                   6
106.5Nm@ 4400rpm                 8
106Nm@ 2200rpm                   3
108.5Nm@ 5000rpm                14
108Nm@ 4400rpm                  20
109Nm@ 4500rpm                  40
11.2@ 4,400(kgm@ rpm)            3
11.3kgm@ 4700rpm                 3
11.4 kgm at 4,000 rpm           30
11.4@ 4,000(kgm@ rpm)            1
11.5@ 4,500(kgm@ rpm)            9
11.8@ 3,200(kgm@ rpm)            1
110(11.2)@ 4800                  1
110@ 3,000(kgm@ rpm)             1
110Nm@ 3000-4300rpm              6
110Nm@ 3000rpm                   1
110Nm@ 3750rpm                  20
110Nm@ 4000rp

In [76]:
## Notes
### So there is no easy way to convert Torque into categorical value or numerical value
### One thing we can do is engineer a new feature called Horse Power from torque columns values.
### Horse power is basically Horsepower = Torque x RPM / 5,252 and we have both torque and rpm values in torque column.  
#### This could be tough because the values formats are not consistent across all rows. 
### For now we'll just ignore the column and revisit it later. 

In [77]:
## Creating the model


In [114]:
## Lets start by create a new dataframe with only the columns we are interested in. 
# model_columns = ['year',  'km_driven', 'mileage', 'engine', 'max_power', 'top_brand', 'cng', 'diesel', 'lpg', 'petrol', 'individual_dealer', 'manual_transmission', 'one_owner', 'two_owners', 'above_three_owners', 'selling_price']
## removing mileage cause km_driven and mileage are sthe same thing. 
car_details = car_details[['year',  'km_driven',  'engine', 'max_power', 'top_brand', 'cng', 'diesel', 'lpg', 'petrol', 'individual_dealer', 'manual_transmission', 'one_owner', 'two_owners', 'above_three_owners', 'selling_price']].copy()


In [115]:
## lets check if correlations have updated and show us any additional insights
car_details.corr()

,year,km_driven,engine,max_power,top_brand,cng,diesel,lpg,petrol,individual_dealer,manual_transmission,one_owner,two_owners,above_three_owners,selling_price
year,1.000000,-0.428548,0.018263,0.226598,-0.011346,0.029095,0.037536,-0.059887,-0.034351,-0.243729,-0.249002,0.491934,-0.317328,-0.339192,0.412302
km_driven,-0.428548,1.000000,0.206031,-0.038159,0.011959,-0.005432,0.271662,0.023095,-0.274460,0.202851,0.201186,-0.295470,0.209913,0.173600,-0.222158
engine,0.018263,0.206031,1.000000,0.703975,-0.125734,-0.059850,0.507383,-0.056681,-0.491189,-0.132215,-0.283206,-0.015510,0.020809,-0.006156,0.455682
max_power,0.226598,-0.038159,0.703975,1.000000,-0.151215,-0.069644,0.304787,-0.060175,-0.286090,-0.286493,-0.541870,0.114942,-0.085206,-0.062002,0.749674
top_brand,-0.011346,0.011959,-0.125734,-0.151215,1.000000,0.008122,-0.045186,0.006656,0.043075,0.019596,0.059533,-0.023090,0.020229,0.007600,-0.120413
cng,0.029095,-0.005432,-0.059850,-0.069644,0.008122,1.000000,-0.088831,-0.005426,-0.072894,0.036808,0.031686,-0.004295,0.009838,-0.008062,-0.033197
diesel,0.037536,0.271662,0.507383,0.304787,-0.045186,-0.088831,1.000000,-0.072800,-0.978020,0.003567,0.025579,-0.040051,0.051721,-0.012760,0.204831
lpg,-0.059887,0.023095,-0.056681,-0.060175,0.006656,-0.005426,-0.072800,1.000000,-0.059739,0.025092,0.025967,-0.024475,0.013442,0.020534,-0.035978
petrol,-0.034351,-0.274460,-0.491189,-0.286090,0.043075,-0.072894,-0.978020,-0.059739,1.000000,-0.012914,-0.034257,0.044107,-0.055230,0.011356,-0.195074
individual_dealer,-0.243729,0.202851,-0.132215,-0.286493,0.019596,0.036808,0.003567,0.025092,-0.012914,1.000000,0.378698,-0.229695,0.168052,0.127362,-0.386151


In [116]:
## Notes
### Its interesting to see year, max_power, have strong positive correlation on selling price
### Also I am surpriced to see negative correlation between manual transmisison and selling price. Car buying patterns have changed in India. 
### If a car has only one owner it has positive impact on selling price, which makes sense. 
### But any more than one owners has a negative impact which intuitively makes sense. 
### Another interesting metric is if fuel type is diesel is has a positive impact on car price but petrol cars reduce the price, which slightly is counter intuitive with max power. 
#### My understanding is diesel cars are not as power ful as petrol cars. 

In [117]:
## Creating the model
### For the first version of the model we are going to use all the above indicators. and se how it performs. 
### We will then remove some indicators and do some more feature engineering if needed to further improve our model. 

In [118]:
### Model v1 Building

In [119]:
## Before we build the model we'll need to scale certain values
## May be we can scale km_driven, mileage, engine & max_power
columns_to_scale = ['km_driven', 'max_power', 'engine']

In [120]:
std_scaler = StandardScaler()
std_scaler

StandardScaler()

In [121]:
scaled_columns = std_scaler.fit_transform(car_details[columns_to_scale])
scaled_columns[:2]


array([[ 1.34377674, -0.49202393, -0.41818825],
       [ 0.89474378,  0.33382687,  0.07798015]])

In [122]:
# convert those results into a dataframe
scaled_df = pd.DataFrame(scaled_columns, columns=columns_to_scale)
scaled_df.head()


,km_driven,max_power,engine
0,1.343777,-0.492024,-0.418188
1,0.894744,0.333827,0.077980
2,1.246926,-0.380120,0.075995
3,1.018008,-0.044408,-0.124457
4,0.894744,-0.094765,-0.318955


In [123]:
print('scaled dataframe',scaled_df.shape)
print('car details',car_details.shape)


scaled dataframe (7906, 3)
car details (7906, 15)


In [124]:
## Dropping original columns
car_details = car_details.drop(columns=columns_to_scale, axis=1)

print('scaled dataframe',scaled_df.shape)
print('car details',car_details.shape)

scaled dataframe (7906, 3)
car details (7906, 12)


In [125]:
car_details

,year,top_brand,cng,diesel,lpg,petrol,individual_dealer,manual_transmission,one_owner,two_owners,above_three_owners,selling_price
0,2014,1,0,1,0,0,1,1,1,0,0,450000
1,2014,1,0,1,0,0,1,1,0,1,0,370000
2,2006,1,0,0,0,1,1,1,0,0,1,158000
3,2010,1,0,1,0,0,1,1,1,0,0,225000
4,2007,1,0,0,0,1,1,1,1,0,0,130000
...,...,...,...,...,...,...,...,...,...,...,...,...
7901,2013,1,0,0,0,1,1,1,1,0,0,320000
7902,2007,1,0,1,0,0,1,1,0,0,1,135000
7903,2009,1,0,1,0,0,1,1,1,0,0,382000
7904,2013,1,0,1,0,0,1,1,1,0,0,290000


In [126]:
## Adding new scalled columns
car_details = pd.concat([car_details.reset_index(drop=True), scaled_df.reset_index(drop=True)], axis=1)

In [127]:
## Declare target variable
y = car_details["selling_price"]
y.shape

(7906,)

In [128]:
# Declare the features
X = car_details.drop(['selling_price'], axis=1)
X.shape

(7906, 14)

In [129]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state=42)

In [130]:
# Instantiate the SKlearn algorithm
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
mymodel = LinearRegression(fit_intercept=True, 
                           normalize=False, 
                           copy_X=True, 
                           n_jobs=None, 
                           positive=False)

In [131]:
# Fit the model to the training dataset
mymodel.fit(X_train, y_train)

/home/gaurangvdave/.local/lib/python3.9/site-packages/sklearn/linear_model/_base.py:148: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.



LinearRegression(normalize=False)

In [132]:
print(mymodel)

LinearRegression(normalize=False)


In [133]:
# what is the intercept? (the value when all predictors are zero)
mymodel.intercept_

-66008680.43418211

In [134]:
# what are the coefficients?
mymodel.coef_

array([  33466.34557444, -159585.32891753,    6334.53141456,
         -1050.45909382,  127369.64299383, -132653.71531458,
       -170912.69985961, -451927.61788661,   19636.6125773 ,
        -29392.06101269,    9755.44843539,  -85245.08193811,
        461472.27233255,  -30653.9154878 ])

In [135]:
# what is the equation for this mymodel?
cols=list(X.columns)
coefs=list(mymodel.coef_)
xcoefs = [(coefs[i], cols[i]) for i in range(0, len(cols))]
output = f'y = {round(mymodel.intercept_,2)} + {round(xcoefs[0][0],2)}*{xcoefs[0][1]} + {round(xcoefs[1][0],2)}*{xcoefs[1][1]} + {round(xcoefs[2][0],2)}*{xcoefs[2][1]} + {round(xcoefs[3][0],2)}*{xcoefs[3][1]} + {round(xcoefs[4][0],2)}*{xcoefs[4][1]} + {round(xcoefs[5][0],2)}*{xcoefs[5][1]} + {round(xcoefs[6][0],2)}*{xcoefs[6][1]}  + {round(xcoefs[7][0],2)}*{xcoefs[7][1]} + {round(xcoefs[8][0],2)}*{xcoefs[8][1]} '
print("Regression Equation: ", output) 

Regression Equation:  y = -66008680.43 + 33466.35*year + -159585.33*top_brand + 6334.53*cng + -1050.46*diesel + 127369.64*lpg + -132653.72*petrol + -170912.7*individual_dealer  + -451927.62*manual_transmission + 19636.61*one_owner 


In [136]:
# add the intercept to the list of coefficients, for display purposes
coefs=list(mymodel.coef_)
cols=list(X.columns)

In [137]:
# turn those into a dataframe.
results = pd.DataFrame(list(zip(cols, coefs)), columns=['feature','coeff'])
results 

,feature,coeff
0,year,33466.345574
1,top_brand,-159585.328918
2,cng,6334.531415
3,diesel,-1050.459094
4,lpg,127369.642994
5,petrol,-132653.715315
6,individual_dealer,-170912.699860
7,manual_transmission,-451927.617887
8,one_owner,19636.612577
9,two_owners,-29392.061013


In [138]:
# show your results as a bar chart
import plotly.express as px
fig = px.bar(x=results['feature'], y=results['coeff'])
fig.update_layout(
    yaxis=dict( title='coefficient'),
    xaxis=dict( title='predictor'),
)
fig.show()

In [139]:
## Notes
### Something funky going on with manual_transmisison 

In [140]:
### Make Predictions

In [141]:
# Predict the y-values on the testing dataset
y_preds = mymodel.predict(X_test)

In [142]:
# Take a look at the first few predictions and compare to the true values
[round(x,2) for x in y_preds[:5]]

[587160.75, 386668.52, 63466.78, 469766.77, 1053797.49]

In [143]:
# Compare that to the actual runs of the first five players
list(y_test[:5])

[501000, 440000, 140000, 476999, 620000]

In [144]:
# compare predictions to known values
import plotly.express as px
fig = px.scatter(x=y_preds, y=y_test, trendline="ols", width=500, height=500)
fig.update_layout(title = 'Evaluation', # Graph title
    xaxis = dict(title = 'predicted values'), # x-axis label
    yaxis = dict(title = 'true values'), # y-axis label   
)
fig.show()

In [109]:
### Evaluate the model

In [110]:
# root mean sq error
rmse = sqrt(metrics.mean_squared_error(y_test, y_preds))
rmse

480685.1710654696

In [111]:
avg_val = y_train.mean()
avg_val

648145.1673132062

In [112]:
# R-2 (coefficient of determination)
r2 = metrics.r2_score(y_test, y_preds)
round(r2, 2)

0.68